In [77]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss

In [78]:
CHUNK_SIZE = 300          # размер чанка в словах
CHUNK_OVERLAP = 50        # перекрытие между чанками
MAX_FEATURES = 20000      # размер словаря TF-IDF
MAX_DF = 0.95             # игнорировать очень частые термы
NGRAM_RANGE = (1, 2)      # униграммы + биграммы
TOP_K_CHUNKS = 20         # сколько кандидатов чанков искать на вопрос
TOP_N_WEBIDS = 5          # сколько web_id отдавать в submit

WEBSITES_PATH = "websites.csv"
QUESTIONS_PATH = "questions_clean.csv"
SUBMIT_PATH = "submit.csv"

In [79]:
def clean_text(text: str) -> str:
    """Очистка текста"""
    text = str(text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def chunk_text(text: str, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    """Разбиваем текст на чанки с перекрытием"""
    words = str(text).split()
    if not words:
        return []
    chunks = []
    step = max(chunk_size - overlap, 1)
    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i+chunk_size])
        if chunk.strip():
            chunks.append(chunk)
    return chunks

Загрузка данных

In [80]:
websites_df = pd.read_csv(WEBSITES_PATH)
questions_df = pd.read_csv(QUESTIONS_PATH)

required_web_cols = {"web_id", "text"}
required_q_cols = {"q_id", "query"}
assert required_web_cols.issubset(set(websites_df.columns)), f"В websites.csv нет колонок: {required_web_cols - set(websites_df.columns)}"
assert required_q_cols.issubset(set(questions_df.columns)), f"В questions_clean.csv нет колонок: {required_q_cols - set(questions_df.columns)}"

print(f"Страниц: {len(websites_df)}, Вопросов: {len(questions_df)}")

Страниц: 1937, Вопросов: 6977


Чанкование

In [81]:
chunk_rows = []
for _, row in websites_df.iterrows():
    web_id = row["web_id"]
    raw_text = row["text"]
    cleaned = clean_text(raw_text)
    chunks = chunk_text(cleaned)
    for ch in chunks:
        chunk_rows.append({"web_id": web_id, "chunk_text": ch})

chunks_df = pd.DataFrame(chunk_rows)
print(f"Получено чанков: {len(chunks_df)} (из {len(websites_df)} страниц)")

# фильтруем пустые чанки и слишком короткие
if len(chunks_df) == 0:
    raise ValueError("После чанкования нет ни одного чанка. Проверь содержимое websites.csv['text'].")

chunks_df["chunk_text_len"] = chunks_df["chunk_text"].str.len()
chunks_df = chunks_df[chunks_df["chunk_text_len"] > 20].drop(columns=["chunk_text_len"])  # минимум 20 символов
print(f"После фильтра: {len(chunks_df)} чанков")

Получено чанков: 8008 (из 1937 страниц)
После фильтра: 7987 чанков


In [82]:
chunk_texts = chunks_df["chunk_text"].tolist()
question_texts = questions_df["query"].fillna("").apply(clean_text).tolist()

# защита от пустых вопросов
question_texts = [q for q in question_texts if q.strip()]
if len(question_texts) != len(questions_df):
    print(f"Найдены пустые вопросы. Останется {len(question_texts)} из {len(questions_df)}")
    question_texts = questions_df["query"].fillna("placeholder").apply(clean_text).tolist()

print(f"{len(chunk_texts)} текстов чанков, {len(question_texts)} вопросов")

7987 текстов чанков, 6977 вопросов


In [83]:
vectorizer = TfidfVectorizer(
    max_features=MAX_FEATURES,
    min_df=1,
    max_df=MAX_DF,
    ngram_range=NGRAM_RANGE,
    stop_words=None
)

chunk_matrix = vectorizer.fit_transform(chunk_texts)          # (num_chunks, vocab_size)
question_matrix = vectorizer.transform(question_texts)        # (num_questions, vocab_size)
print(f"Матрица чанков: {chunk_matrix.shape}, Матрица вопросов: {question_matrix.shape}")

#float32 + L2-нормализация для косинусного сходства в IP
chunk_dense = chunk_matrix.astype(np.float32).toarray()
question_dense = question_matrix.astype(np.float32).toarray()
faiss.normalize_L2(chunk_dense)
faiss.normalize_L2(question_dense)

Матрица чанков: (7987, 20000), Матрица вопросов: (6977, 20000)


In [84]:
dim = chunk_dense.shape[1]

# создаём HNSW индекс
M = 32   # количество связей
index = faiss.IndexHNSWFlat(dim, M)

# нормализация векторов для косинусного сходства
faiss.normalize_L2(chunk_dense)
index.add(chunk_dense)

print(f"Индекс HNSW готов, Векторов: {index.ntotal}, размерность: {dim}")

# поиск
distances, indices = index.search(question_dense, TOP_K_CHUNKS)
print("Поиск завершён (HNSW)")

Индекс HNSW готов, Векторов: 7987, размерность: 20000
Поиск завершён (HNSW)


In [85]:
results = []
for i, q_id in enumerate(questions_df["q_id"]):
    # индексы найденных чанков
    candidate_web_ids = []
    for idx in indices[i]:
        if idx == -1:
            continue
        if 0 <= idx < len(chunks_df):
            wid = chunks_df.iloc[idx]["web_id"]
            if pd.notna(wid) and str(wid).strip() != "":
                candidate_web_ids.append(str(wid).strip())

    # удаляем дубликаты
    seen = set()
    unique_web_ids = []
    for wid in candidate_web_ids:
        if wid not in seen:
            seen.add(wid)
            unique_web_ids.append(wid)

    # сокращаем до top_n_webids и дополняем пустыми строками
    unique_web_ids = unique_web_ids[:TOP_N_WEBIDS]
    while len(unique_web_ids) < TOP_N_WEBIDS:
        unique_web_ids.append("")

    results.append({
        "q_id": q_id,
        "web_id_1": unique_web_ids[0],
        "web_id_2": unique_web_ids[1],
        "web_id_3": unique_web_ids[2],
        "web_id_4": unique_web_ids[3],
        "web_id_5": unique_web_ids[4],
    })

submission_df = pd.DataFrame(results)

# валидации формата
assert list(submission_df.columns) == ["q_id","web_id_1","web_id_2","web_id_3","web_id_4","web_id_5"], "Формат колонок неверный"
assert len(submission_df) == len(questions_df), "Количество строк не совпадает с количеством вопросов"

submission_df.to_csv(SUBMIT_PATH, index=False)
print(f"Файл сохранён: {SUBMIT_PATH}")

Файл сохранён: submit.csv


Статистика submit.csv

In [86]:
total_nonempty = int((submission_df[["web_id_1","web_id_2","web_id_3","web_id_4","web_id_5"]] != "").sum().sum())
avg_per_q = total_nonempty / len(submission_df)
unique_wids = pd.unique(submission_df[["web_id_1","web_id_2","web_id_3","web_id_4","web_id_5"]].values.ravel())
unique_wids = [w for w in unique_wids if isinstance(w, str) and w.strip() != ""]
print(f"- Вопросов: {len(submission_df)}")
print(f"- Уникальных web_id в submit: {len(set(unique_wids))}")
print(f"- Всего непустых ответов: {total_nonempty}")
print(f"- Среднее web_id на вопрос: {avg_per_q:.2f}")

- Вопросов: 6977
- Уникальных web_id в submit: 1446
- Всего непустых ответов: 34325
- Среднее web_id на вопрос: 4.92
